In [ ]:
# Step 1: Install necessary libraries
!pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib

# Step 2: Import required libraries
import os
import pickle
import base64
import time
import json
from google.colab import drive
from googleapiclient.discovery import build
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request

# Step 3: Mount Google Drive (to store the token file)
drive.mount('/content/drive')

# Step 4: Define the credentials path and scope
TOKEN_PATH = '/content/drive/MyDrive/token.json'
CLIENT_SECRET_FILE = '/content/drive/MyDrive/credentials.json'
SCOPES = ['https://www.googleapis.com/auth/gmail.readonly']

# Step 5: Authenticate and get credentials
def authenticate_gmail():
    """Authenticate the user and get the Gmail API service."""
    creds = None

    # Check if token.json exists, otherwise authenticate
    if os.path.exists(TOKEN_PATH):
        creds = Credentials.from_authorized_user_file(TOKEN_PATH, SCOPES)

    # If there are no valid credentials, authenticate
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(CLIENT_SECRET_FILE, SCOPES)
            auth_url, _ = flow.authorization_url(prompt='consent')

            print("Please go to this URL to authenticate: ", auth_url)
            auth_code = input("Enter the authorization code: ")

            creds = flow.fetch_token(authorization_response=f"http://localhost/?code={auth_code}")

        # Save the credentials for future use
        with open(TOKEN_PATH, 'w') as token:
            token.write(creds.to_json())

    # Build the Gmail service
    service = build('gmail', 'v1', credentials=creds)
    return service

# Step 6: List messages with 'is:spam' query
def list_spam_messages(service):
    """List Gmail messages with the 'is:spam' label."""
    try:
        results = service.users().messages().list(userId='me', labelIds=['SPAM'], q="is:spam").execute()
        messages = results.get('messages', [])

        if not messages:
            print("No spam messages found.")
        else:
            print(f"Found {len(messages)} spam messages.")
            return messages
    except Exception as error:
        print(f"An error occurred: {error}")
        return []

# Step 7: Get message details (e.g., subject, snippet)
def get_message_details(service, msg_id):
    """Get message details such as subject, snippet, etc."""
    try:
        msg = service.users().messages().get(userId='me', id=msg_id).execute()
        payload = msg['payload']
        headers = payload['headers']

        subject = ''
        snippet = msg.get('snippet', '')

        for header in headers:
            if header['name'] == 'Subject':
                subject = header['value']

        print(f"Subject: {subject}")
        print(f"Snippet: {snippet}")

        return subject, snippet
    except Exception as error:
        print(f"An error occurred: {error}")

# Step 8: Process the spam emails
def process_spam_messages(service):
    """Process and print details of spam messages."""
    messages = list_spam_messages(service)

    for message in messages:
        msg_id = message['id']
        get_message_details(service, msg_id)

# Step 9: Run automation every 10 minutes
def automate_spam_detection():
    """Run the automation to detect and process spam messages."""
    print("Starting Gmail spam detection...")

    # Authenticate and create the Gmail API service
    service = authenticate_gmail()

    # Process spam messages
    process_spam_messages(service)

# Step 10: Run the automation every 10 minutes
if __name__ == '__main__':
    while True:
        automate_spam_detection()
        print("Waiting for the next cycle...")
        time.sleep(600)  # Sleep for 10 minutes


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 50.8 MB/s eta 0:00:00
  Attempting uninstall: google-api-python-client
    Found existing installation: google-api-python-client 2.155.0
    Uninstalling google-api-python-client-2.155.0:
      Successfully uninstalled google-api-python-client-2.155.0
Mounted at /content/drive
Starting Gmail spam detection...
Please go to this URL to authenticate:  https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=922208649282-c6vpr6822pkt1pq0704g36g9a2p2t79g.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fgmail.readonly&state=O4mX3oOqykzhuvrxW41j9UZj2kX5BU&prompt=consent&access_type=offline
